In [ ]:
# upload elife dataset, as well as api_key.txt
from google.colab import files
files.upload()

{}

In [ ]:
!pip install openai
import openai
import pandas as pd
from zipfile import ZipFile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.0 MB/s eta 0:00:00


In [ ]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # file is a single line w/api key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

def loadDataset(path, fileName):
  zip_file = ZipFile(path)
  return pd.read_json(zip_file.open(fileName))

# Set up OpenAI API client
openai.api_key = get_file_contents('api_key.txt')
df = loadDataset('/content/elife.zip', 'test.json')
df

'api_key.txt' file not found


In [ ]:
# add by Jiarui Liu, chunk sections in a single paper into chunk list. chunk are separated between sentence (i.e. a sentence won't be separated into difference chunks unless the length of this single
# sentence is higher than chun_size. given max_token = 4096, this is unlikely to happen)
#does NOT work if paper's length is smaller than chunk_size
import re

def paper_to_chunks(single_paper_sections, chunk_size):
  
  def split_list_into_chunks(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

  # get [list of [list of tokens in one sentence] ]
  tokenedSentenceListNoCut = [re.split('\s+', sentence)  for section in single_paper_sections for sentence in section]
  tokenedSentenceList = []
  # ensure each sentence has length under chunk_size
  for tokenedSentence in tokenedSentenceListNoCut:
    if len(tokenedSentence) < chunk_size:
      for chunk in split_list_into_chunks(tokenedSentence, chunk_size):
        tokenedSentenceList.append(chunk)
    else:
      tokenedSentenceList.append(tokenedSentence)


  # generate chunks
  chunkLst = []

  singleChunk = []
  c = 0
  for tokenedSentence in tokenedSentenceList:
    if c + (length := len(tokenedSentence)) < chunk_size: # add sentence into recent chunk
        singleChunk.append(tokenedSentence)
        c += length
    else: # chunk full, flatten the chunk, add it into chunkLst and create new chunk
      chunkLst.append([token for sentence in singleChunk for token in sentence])
      singleChunk = [tokenedSentence]
      c = length

  return chunkLst

NameError: ignored

In [ ]:
def complete_chunk_summary(chunk_text, reply_summary = ''):
  #given a chunk, summarize it

  #chunk_text is the original text to be summarized
  #reply_summary is all of the summary that has been written thus far
  response = openai.Completion.create(
              engine="text-davinci-003",
              #our gpt prompt
              prompt=f"Briefly summarize the following in simple terms:\n{chunk_text}\nSummary:{reply_summary}",
              temperature=0.7,
              max_tokens=50,
              n=1,
              stop=None,
              timeout=15,
  )
  new_reply_summary = reply_summary + ' ' + response.choices[0].text.strip()
  finish = response.choices[0].finish_reason
  if finish == 'stop':
    return new_reply_summary
  elif finish == 'length':
    return complete_chunk_summary(chunk_text, reply_summary = new_reply_summary)
  else:
    print(f'Error: returned response choice that wasnt "stop" or "length", it was {finish}')

In [ ]:
#To be used only in the case of a paper which is shorter than chunk_size

p = df.iloc[['index here']]
#gpt_summary()
for chunk in paper_to_chunks(p['sections'].squeeze(), 800):
  chunk_text = ' '.join(chunk)

  # get the summary for the chunk
  chunk_summary = complete_chunk_summary(chunk_text)

  print(chunk_summary)

In [ ]:
#testing the summary on a single paper

completions = openai.Completion()
paper_chunks = paper_to_chunks(df.sections[0], 2500)
summary = ''
for chunk in paper_chunks:
  chunk_text = ' '.join(chunk)
  summary += complete_chunk_summary(chunk_text) + " "
print(summary)

 Movement sequences are composed of a series of movements that can be learned through practice. The cerebellum is responsible for learning and executing accurate movements and is particularly sensitive to dysdiadochokinesia which is an inability to perform a rapid alternating sequence of movements. Studies of sequence learning in non-human primates have demonstrated that with repeated training, components of the sequence can be initiated predictively before the arrival of the sensory cue. Experiments were done to test the hypothesis that the cerebell um can use feedback signals from a movement as the cue for the next movement in a sequence. Results showed that feedback information about the first movement was sufficient for the cerebellum to use as a new cue for the subsequent movement and that the second movement was acquired through standard cerebellar-dependent associative learning. On some trials, a third and fourth movement were also observed.  A study was conducted to determine i

In [ ]:
def gpt_summary(paper):
  #get the text of the section
  section_text = paper['sections'].squeeze()

  #break the section into chunks
  paper_chunks = paper_to_chunks(section_text, 2300)
    
  # iterate over each chunk in the section
  for chunk in paper_chunks:
        chunk_text = ' '.join(chunk)

        # get the summary for the chunk
        chunk_summary = complete_chunk_summary(chunk_text)

        return(chunk_summary)

In [ ]:
!pip install tqdm
import json
import os
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#given a folder path, return the index of the most recent pickled checkpoint
def most_recent_pickle_index(save_folder_path):
  filenames = os.listdir(save_folder_path)

  # Filter filenames that match the pattern "checkpoint_{index}.pkl"
  filtered_filenames = [filename for filename in filenames if filename.startswith('checkpoint_') and filename.endswith('.pkl')]

  # Extract the indices from the filenames and convert them to integers
  indices = [int(filename.split('_')[1].split('.')[0]) for filename in filtered_filenames]

  # Check if there are any filenames
  if indices:
      # Find the filename with the highest index
      return max(indices)
  else:
      return None

In [ ]:
def summarize_with_checkpoint(path, JSONfileName, summary_function, nameOfYourSummary, save_folder_path, **kwargs):
  '''
  path: path to original dataset
  JSONfileName: name of original dataset
  summary_function: takes a (1, n) row of the original dataset as input, returns a summary string
  nameOfYourSummary: the name to save the new summary under in the final dataframe
  
  save_folder_path: path to where to save/retrieve files, e.g. "/content/pickledfiles"
  # WARNING: DON'T USE UNDERSCORES IN SAVE_FOLDER_PATH #

  '''
  recent_index = most_recent_pickle_index(save_folder_path)
  print(f'recent index is {recent_index}')
  recent_path = f'{save_folder_path}checkpoint_{recent_index}.pkl'
  
  original_dataset = loadDataset(path, JSONfileName)

  if recent_index == None:
    #first time in:
    our_summary = pd.DataFrame()
    #set up the correct colums
    cols_to_copy = ['id', 'year', 'title', 'abstract', 'summary', 'keywords']
    for col_name in cols_to_copy:
      our_summary[col_name] = ''
    our_summary[nameOfYourSummary] = ''
    recent_index = -1
  
  else:
    our_summary = pd.read_pickle(recent_path)

  for index, row in original_dataset.iloc[recent_index+1:].iterrows():
    paper = pd.Series(row).copy().to_frame().T
    #call the summary function on the paper
    generated_summary = summary_function(paper, **kwargs)
    paper[nameOfYourSummary] = generated_summary

    our_summary = pd.concat([our_summary, paper])

    print(f"pickling index {index}")
    our_summary.to_pickle(f'{save_folder_path}checkpoint_{index}.pkl')

In [ ]:
import time

def test(paper):
  time.sleep(1)
  return paper['id']

summarize_with_checkpoint(path = '/content/elife.zip', JSONfileName = 'test.json', summary_function = test_summary_function, nameOfYourSummary = 'gpt', save_folder_path = '/content/test_summaries/')

recent index is 2
pickling index 3
pickling index 4
pickling index 5


KeyboardInterrupt: ignored

In [ ]:
##################
# the main call  #
##################
summarize_with_checkpoint(path = '/content/elife.zip', JSONfileName = 'test.json', summary_function = gpt_summary, nameOfYourSummary = 'gpt', save_folder_path = '/content/gptsummarypickle/')

In [ ]:
import glob
import pickle

def dumpLastPklToJson(path, jsonFileName, jsonFileStoringPath):
    # Get list of all pkl files in the directory
    pkl_files = glob.glob(os.path.join(path, 'checkpoint_*.pkl'))

    # If there are no pkl files, return
    if not pkl_files:
        print("No pkl files found")
        return

    # Sort the files by modification time
    pkl_files.sort(key=os.path.getmtime)

    # Get the latest file
    latest_pkl_file = pkl_files[-1]

    # Load data from the latest pkl file
    with open(latest_pkl_file, 'rb') as f:
        data = pickle.load(f)

    # Dump data to JSON file
    json_file_path = os.path.join(jsonFileStoringPath, jsonFileName)
    if isinstance(data, pd.DataFrame):
        # If data is a DataFrame, use pandas' to_json() method
        data.to_json(json_file_path, orient="records", lines=True)
    else:
        # If data is not a DataFrame, use the json module
        with open(json_file_path, 'w') as f:
            json.dump(data, f, indent=4)

    print(f"Data from {latest_pkl_file} has been dumped to {json_file_path}")

In [ ]:
dumpLastPklToJson('/content/', 'gpt_summaries', '/content/')

Data from /content/checkpoint_240.pkl has been dumped to /content/gpt_summaries


In [ ]:
fromJSON = pd.read_json('test_json_name', lines = True)
fromJSON

ValueError: ignored